# Unit 5 - Financial Planning

In [ ]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import warnings

%matplotlib inline

In [ ]:
# Performance Warnings management
warnings.filterwarnings('ignore', message='DataFrame has High Levels of Fragmentation', category=pd.errors.PerformanceWarning)


In [ ]:
# Load .env environment variables
load_dotenv()

# Check if API keys are loaded correctly
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

if not alpaca_api_key or not alpaca_secret_key:
    raise ValueError("API keys not found. Please check your .env file.")
else:
    print("API keys loaded successfully.")


## Part 1 - Personal Finance Planner

### Collect Crypto Prices Using the `requests` Library

In [ ]:
# Set current amount of crypto assets
my_btc = 1.2  # Example: owning 1.2 Bitcoins
my_eth = 5.3  # Example: owning 5.3 Ethereum


In [ ]:
# Crypto API URLs
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=USD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=USD"


In [ ]:
# Initial import
import requests

# Fetch current BTC price
btc_response = requests.get(btc_url)
btc_data = btc_response.json()
btc_price = btc_data['data']['1']['quotes']['USD']['price']

# Fetch current ETH price
eth_response = requests.get(eth_url)
eth_data = eth_response.json()
eth_price = eth_data['data']['1027']['quotes']['USD']['price']

# Compute current value of my crpto
my_btc_value = my_btc * btc_price
my_eth_value = my_eth * eth_price

# Print current crypto wallet balance
print(f"The current value of your {my_btc} BTC is ${my_btc_value:0.2f}")
print(f"The current value of your {my_eth} ETH is ${my_eth_value:0.2f}")


### Collect Investments Data Using Alpaca: `SPY` (stocks) and `AGG` (bonds)

In [ ]:
# Set current amount of shares
my_agg = 200  # number of shares of AGG
my_spy = 50   # number of shares of SPY


In [ ]:
# Initial import
import os
from alpaca_trade_api import REST

# Set Alpaca API key and secret
# Set Alpaca API key and secret from environment variables
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')

# Checking if the API key and secret are available
if alpaca_api_key is None or alpaca_secret_key is None:
    raise Exception("Alpaca API key or secret not found. Check your .env file.")

# Create the Alpaca API object
# Creating the Alpaca API object
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)


In [ ]:
# Format current date as ISO format
start_date = pd.Timestamp("2020-08-07", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2020-08-07", tz="America/New_York").isoformat()

# Set the tickers
tickers = ["AGG", "SPY"]

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get current closing prices for SPY and AGG
df_ticker = api.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

# Reorganize the DataFrame
# Separated ticker data
AGG = df_ticker[df_ticker["symbol"]=="AGG"].drop("symbol", axis=1)
SPY = df_ticker[df_ticker["symbol"]=="SPY"].drop("symbol", axis=1)


# Concatenate the ticker DataFrames
df_ticker = pd.concat([AGG, SPY], axis=1, keys=["AGG","SPY"])

# Preview DataFrame
df_ticker


In [ ]:
# Pick AGG and SPY close prices
agg_close_price = df_ticker["AGG"]["close"].values[0]
spy_close_price = df_ticker["SPY"]["close"].values[0]

# Print AGG and SPY close prices
print(f"Current AGG closing price: ${agg_close_price}")
print(f"Current SPY closing price: ${spy_close_price}")


In [ ]:
# Compute the current value of shares
my_spy_value = my_spy * spy_close_price
my_agg_value = my_agg * agg_close_price

# Print current value of shares
print(f"The current value of your {my_spy} SPY shares is ${my_spy_value:0.2f}")
print(f"The current value of your {my_agg} AGG shares is ${my_agg_value:0.2f}")


### Savings Health Analysis

In [ ]:
# Set monthly household income
monthly_income = 12000

# Consolidate financial assets data
total_crypto_savings = my_btc_value + my_eth_value
total_shares_savings = my_spy_value + my_agg_value

# Create savings DataFrame
data = {'amount': [total_crypto_savings, total_shares_savings]}
df_savings = pd.DataFrame(data, index=['crypto', 'shares'])

# Display savings DataFrame
display(df_savings)


In [ ]:
# Plot savings pie chart
df_savings['amount'].plot(kind='pie', autopct='%1.1f%%', startangle=120, title="Savings Pie Chart", legend='upper right')

In [ ]:
# Set ideal emergency fund
emergency_fund = monthly_income * 3

# Calculate total amount of savings
total_savings = total_crypto_savings + total_shares_savings

# Validate saving health
if total_savings > emergency_fund:
    print("Congratulations! Your emergency fund has optimal amount of funds. Total savings: ${total_savings:.2f}")
elif total_savings == emergency_fund:
    print("Congratulations on reaching your financial goal! You have exactly the amount needed for an emergency fund. Total savings: ${total_savings:.2f}")
elif total_savings < emergency_fund:
    print(f"You're so close! You are ${emergency_fund - total_savings} away from the optimal emergency fund amount. Total savings: ${total_savings:.2f}")

## Part 2 - Retirement Planning

### Monte Carlo Simulation

In [ ]:
''' # Set start and end dates of five years back from today.
# Sample results may vary from the solution based on the time frame chosen
import pandas as pd

start_date = pd.Timestamp('2016-05-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2021-05-01', tz='America/New_York').isoformat() '''

import pandas as pd
from datetime import datetime, timedelta

# Calculating today's date
today = datetime.now()

# Setting start and end dates dynamically for five years back from today
start_date = (today - timedelta(days=5*365)).replace(hour=0, minute=0, second=0, microsecond=0).isoformat()
end_date = today.replace(hour=0, minute=0, second=0, microsecond=0).isoformat()


In [ ]:
import pandas as pd
import alpaca_trade_api as tradeapi

# Setting up Alpaca API
api = tradeapi.REST('ALPACA_API_KEY', 'ALPACA_SECRET_KEY', base_url='https://paper-api.alpaca.markets/v2')

# Setting the tickers and timeframe
tickers = ["SPY", "AGG"]
timeframe = '1Day'

# Get 5 years' worth of historical data for SPY and AGG
df_stock_data = api.get_barset(
    tickers,
    timeframe,
    start=start_date,
    end=end_date
).df

# Separating ticker data for clarity
spy_data = df_stock_data['SPY']
agg_data = df_stock_data['AGG']

# Concatenating the ticker DataFrames
df_stock_data = pd.concat([spy_data, agg_data], axis=1, keys=["SPY", "AGG"])

# Displaying sample data
print(df_stock_data.head())


In [ ]:
from MCForecastTools import MCSimulation

# Configuring a Monte Carlo simulation to forecast 30 years cumulative returns
mc_forecast = MCSimulation(
    portfolio_data = df_stock_data,
    weights = [0.6, 0.4],  # 60% stocks (SPY), 40% bonds (AGG)
    num_simulation = 50,
    num_trading_days = 252 * 30
)

# Running the Monte Carlo simulation
mc_forecast.calc_cumulative_return()

# Plotting the simulation outcomes
mc_forecast.plot_simulation()
mc_forecast.plot_distribution()


In [ ]:
# Printing the simulation input data
print("Input Data for Monte Carlo Simulation:")
print(mc_forecast.portfolio_data.head())


In [ ]:
# Running a Monte Carlo simulation to forecast 30 years cumulative returns
mc_results = mc_forecast.calc_cumulative_return()

print("First few simulation outcomes:")
print(mc_results.head())


In [ ]:
# Plot simulation outcomes
mc_forecast.plot_simulation()


In [ ]:
# Plot probability distribution and confidence intervals
import matplotlib.pyplot as plt

# Executing the plot distribution method
mc_forecast.plot_distribution()

# Adding details to enhance the plot
plt.title('Probability Distribution of Cumulative Returns Over 30 Years')
plt.xlabel('Cumulative Returns')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()


### Retirement Analysis

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
summary_stats = mc_forecast.summarize_cumulative_return()

# Print summary statistics
print("Summary Statistics from Monte Carlo Simulation:")
print(summary_stats)


### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `$20,000` initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
ci_lower = initial_investment * summary_stats[0.025]
ci_upper = initial_investment * summary_stats[0.975]

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `50%` increase in the initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000 * 1.5

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $30,000
ci_lower = initial_investment * summary_stats[0.025]
ci_upper = initial_investment * summary_stats[0.975]

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

## Optional Challenge - Early Retirement


### Five Years Retirement Option

In [ ]:
from MCForecastTools import MCSimulation

# Configuring a Monte Carlo simulation to forecast 5 years cumulative returns
mc_five_year = MCSimulation(
    portfolio_data = df_stock_data,
    weights = [0.6, 0.4],
    num_simulation = 500,
    num_trading_days = 252 * 5
)


In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
mc_five_year_results = mc_five_year.calc_cumulative_return()


In [ ]:
# Plot simulation outcomes
print("First few simulation outcomes:")
print(mc_five_year_results.head())


In [ ]:
import matplotlib.pyplot as plt

# Plot probability distribution and confidence intervals
ax = mc_five_year.plot_distribution()

# Customizing the plot
ax.set_title('5-Year Return Distribution with Confidence Intervals')
ax.set_xlabel('Final Portfolio Value')
ax.set_ylabel('Frequency')

lower_bound = mc_five_year_results.quantile(0.05)
upper_bound = mc_five_year_results.quantile(0.95)
plt.axvline(x=lower_bound, color='r', linestyle='--', label='5th Percentile')
plt.axvline(x=upper_bound, color='r', linestyle='--', label='95th Percentile')
plt.legend()

plt.show()


In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
summary_stats = mc_five_year.summarize_cumulative_return()

# Print summary statistics
print("Summary Statistics from the Monte Carlo Simulation:")
print(summary_stats)


In [ ]:
# Set initial investment
initial_investment = 60000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
ci_lower_five = initial_investment * summary_stats[0.025]
ci_upper_five = initial_investment * summary_stats[0.975]

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five} and ${ci_upper_five}")

### Ten Years Retirement Option

In [ ]:
from MCForecastTools import MCSimulation

# Configuring a Monte Carlo simulation to forecast 10 years cumulative returns
mc_ten_year = MCSimulation(
    portfolio_data = df_stock_data,
    weights = [0.6, 0.4],
    num_simulation = 500,
    num_trading_days = 252 * 10
)


In [ ]:
# Running a Monte Carlo simulation to forecast 10 years cumulative returns
mc_ten_year_results = mc_ten_year.calc_cumulative_return()


In [ ]:
# Plot simulation outcomes
mc_ten_year.plot_simulation()


In [ ]:
# Plot probability distribution and confidence intervals
mc_ten_year.plot_distribution()

ax.set_title('Distribution of 10-Year Portfolio Cumulative Returns')
ax.set_xlabel('Cumulative Returns')
ax.set_ylabel('Frequency')
import matplotlib.pyplot as plt
plt.show()


In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
summary_statistics_ten_year = mc_ten_year.summarize_cumulative_return()

# Print summary statistics
print("Summary Statistics from the Monte Carlo Simulation:")
print(summary_statistics_ten_year)


In [ ]:
# Set initial investment
initial_investment = 60000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
ci_lower_ten = initial_investment * summary_statistics_ten_year[0.025]
ci_upper_ten = initial_investment * summary_statistics_ten_year[0.975]

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 10 years will end within in the range of"
      f" ${ci_lower_ten:,.2f} and ${ci_upper_ten:,.2f}")


In [2]:
'''
Future Development list.append

// Step 1: Review Current Implementation
I will start by reviewing the current code to understand its structure and identify any immediate inefficiencies or redundant processes.

// Step 2: Optimize Data Fetching
- I will ensure that data fetching from APIs (like Alpaca) is done efficiently.
- I will check if I am making any unnecessary API calls.
- I will consider implementing caching mechanisms to store historical data, reducing the need to fetch it multiple times.

// Step 3: Efficient Data Handling
- I will review how data is handled and stored. I will replace loops with vectorized operations using pandas or NumPy to enhance performance.
- I will ensure that data transformations are minimized and use in-place operations wherever possible.

// Step 4: Optimize Monte Carlo Simulation
- I will inspect the Monte Carlo simulation setup to identify any potential for reducing computational overhead.
- I will experiment with different numbers of simulations to find a balance between accuracy and performance.
- If the simulations are independent of each other, I will explore parallel processing techniques to run them concurrently.

// Step 5: Memory Management
- I will regularly check for variables that are no longer in use and explicitly free up memory.
- I will use more efficient data types (for example, changing float64 to float32 where precision is not critical).

// Step 6: Code Modularity and Reusability
- I will refactor the code into functions or classes to improve modularity, making it easier to manage and test.
- I will create utility functions for repetitive tasks like fetching data, setting up simulations, and plotting results.

// Step 7: Testing and Validation
- I will write unit tests for each component to ensure reliability.
- I will implement validation checks to ensure the integrity of the data being processed.

// Step 8: Documentation and Comments
- I will document the code thoroughly, including detailed comments on the purpose and logic of each function.
- I will ensure that all parameters and return types are clearly specified.

// Step 9: Performance Profiling
- I will use profiling tools to identify bottlenecks in the code.
- Based on profiling results, I will focus on optimizing the most time-consuming parts of the code.

// Step 10: User Interaction and Error Handling
- I will refine the user interface for inputting parameters and displaying results, making it more user-friendly.
- I will improve error handling to manage unexpected or invalid inputs gracefully.

// Step 11: Future Proofing
- I will keep the code flexible to accommodate new financial models or data sources.
- I will plan for potential future features like adding different types of investments or adjusting the simulation algorithms.

// Step 12: Final Review and Testing
- I will conduct a final review of the entire codebase.
- I will perform comprehensive testing to ensure everything works seamlessly.
- I will gather feedback on the usability and performance of the system.
'''

SyntaxError: incomplete input (3857640367.py, line 1)